# Match Kitti Sequence version 22-11

Reads a sequence of stereo images as Frame. The Frames are scanned to identify FramePoints that are pixels that are at the top or botrom of vertical lines in the image. A patch is taken around each of the FramePoints in the left hand image, to match these to a patch in the right hand image and estimate the disparity/distance of the framePoint.

A Sequence is created in which the first Frame is the KeyFrame. It's initial FramePoints are culled to keep only the points for which there is a confident estimate of its depth. The consecutive frames are used to track the KeyFrame's remaining FramePoints. Only a limited number of all framePoints will be found in the consecutive frames, mostly because nearby objects become larger quickly as we get closer. The matching FramePoints are given the same FramePointId as the corresponding point in the KeyFrame and its depth is estimated.

The results are dumped into a folder 'dump', containing files 0.txt, 1.txt, etc. 0.txt contains the FramePoints as observed from the first frame, 1.txt form the second frame, etc. In these files, each row contains a single FramePoint with the values FramePointID, X, Y, Depth.

In [4]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
os.environ['STEREO_CONFIDENCE'] = '1.6'
os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

from pyurb.urb_kitti import *
%matplotlib inline
import numpy as np

Read the first 10 frames from KITTI sequence 00.

In [5]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_2'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_3'

frame = []
for frameid in range(FRAMECOUNT):
    filename = '%06d.png'%(frameid)
    left_frame = Frame(LEFTDIR + '/' + filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 431 µs, sys: 848 µs, total: 1.28 ms
Wall time: 1.29 ms


In [6]:
%%bash
ls /data/urbinn/datasets/kitti/sequences/01/image_2 | wc -l


1101


Inspect the FramePoints for the first frame

We can also manually compute the Depth of the points and show these

Turn the list of frames into a sequence. The first frame is automatically taken as the KeyFrame and its FramePoints are rerieved, depth is estimated, only the most confident points are kept, and they are tracked in the other frames.

In [ ]:
seq = Sequence()
for f in ProgressBar()(frame):
    seq.add_frame(f)

  0% (5 of 1101) |                        | Elapsed Time: 0:00:07 ETA:  0:26:43

invalid speed 4.20373948795 -1.03510464781 
 [[ 0.97477079  0.0036529  -0.22317834  1.76321253]
 [-0.00251859  0.99998243  0.00536694 -0.1015843 ]
 [ 0.22319402 -0.00466944  0.97476286 -0.01759371]
 [ 0.          0.          0.          1.        ]]


  1% (16 of 1101) |                       | Elapsed Time: 0:00:19 ETA:  0:12:13

150 -1.00338851616 [[  9.98943459e-01   9.72309543e-05  -4.59560289e-02  -2.08941232e-02]
 [ -1.12544650e-05   9.99998249e-01   1.87109748e-03   7.17438652e-03]
 [  4.59561304e-02  -1.86860338e-03   9.98941711e-01  -1.00338852e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
invalid speed -2.08640786322 -1.09959396519 
 [[  9.95641204e-01   3.08054441e-03  -9.32153570e-02   6.39715736e-02]
 [ -2.89525640e-03   9.99993555e-01   2.12291185e-03   3.22341005e-02]
 [  9.32212960e-02  -1.84377615e-03   9.95643707e-01  -2.08640786e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


  1% (17 of 1101) |                       | Elapsed Time: 0:00:22 ETA:  0:15:33

...

  4% (49 of 1101) |#                      | Elapsed Time: 0:01:52 ETA:  1:31:11

invalid speed 4.40441479961 -1.23627740149 
 [[ 0.99888723  0.01217285 -0.04556454  2.50112023]
 [-0.01201411  0.99992077  0.00375614  0.13688259]
 [ 0.04560665 -0.00320454  0.99895434  0.55834975]
 [ 0.          0.          0.          1.        ]]


  4% (52 of 1101) |#                      | Elapsed Time: 0:02:21 ETA:  1:53:51

invalid speed 4.45539006632 -1.33682050348 
 [[  9.99100797e-01   1.46041332e-02  -3.98034864e-02   3.16815026e+00]
 [ -1.47382846e-02   9.99886645e-01  -3.07897970e-03   5.28286593e-01]
 [  3.97540086e-02   3.66284618e-03   9.99202783e-01   3.60216947e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


  4% (53 of 1101) |#                      | Elapsed Time: 0:02:43 ETA:  2:26:06

In [ ]:
show2(draw_frame_depth(frame[0]), draw_frame_depth(frame[1]))

Show only the ID's in frame[0] that are actually in frame[1], to avoid clutter in the image.

In [2]:
#kp0 = [kp for kp in frame[0].get_observations() if kp.get_mappoint() in { p.get_mappoint() for p in frame[1].get_observations() }]
#kp1 = [kp for kp in frame[1].get_observations() if kp.has_mappoint()]
#frame[1].compute_depth()
#show2(draw_observations_depth(kp0), draw_observations_depth(kp1))

In [3]:
frame[5].compute_depth()
frame[5].filter_not_useful()
frame[6].compute_depth()
frame[6].filter_not_useful()
obs1 = [o for o in frame[5].get_observations() ]
obs2 = [o for o in frame[6].get_observations() ]

NameError: name 'frame' is not defined

In [ ]:
show2(draw_observations_depth(obs1), draw_observations_depth(obs2))

In [ ]:
frame[5].compute_depth()
a = [ (o.get_depth(), o.confidence) for o in frame[5].get_observations() if o.confidence is not None and o.confidence > 1.4 ]
len(a)
#show2(draw_frame(frame[5]), draw_frame(frame[5].get_right_frame()))

In [ ]:
pl = [ o for o in frame[5].get_observations() if o.confidence is not None and o.confidence > 1.4 ]
show(draw_observations_depth(pl))

In [ ]:
show2(draw_frame(frame[0]), draw_frame(f.get_right_frame()))

In [ ]:
f = frame[0]

In [ ]:
f.compute_depth()

In [ ]:
show(draw_observations_depth(frame[0].get_observations()[:100]))